# 地图热力图可视化
## 请选择年份
年份的可选值： 2010, 2011, 2012, 2014, 2016, 2018

In [1]:
year = 2010

## 展示table

In [2]:
print([table for table in cfps[year]])

['adult', 'child', 'comm', 'famconf', 'famecon']


### 选择一个table

In [3]:
table = 'child'

In [4]:
schema = cfps[year][table].schema

## 选择其中一列数据，
这里可选变量与您所选择的table、year有关

In [5]:
print("您可选择的变量有:")
for var,value in schema.items():
    if value['type'] in ['int32','float64']: 
        print(f'{var}:{value["key"]}')

您可选择的变量有:
pid:个人id
fid:家庭样本代码
cid:村居样本代码
countyid:区县顺序码
psu:primary sampling unit
rswt_nat:个人权重-全国样本
rswt_res:个人权重-全国再抽样样本
indno:个人的家庭成员顺序号
cyear:采访年份
cmonth:采访月份
wa1y:请问孩子的出生日期（年）
wa1m:请问孩子的出生日期（月）
wa1age:孩子的年龄（年）
wa1month:孩子的年龄（月）
wa101:孩子的胎龄（月）
wa102:孩子出生时的体重（斤）
wa103:孩子现在的体重（斤）
wa104:孩子现在的身高（厘米）
wa105:到现在为止，孩子吃母乳有多长时间（月）
wa107c_code:孩子的出生地（县/区编码）
wa501c_code:孩子现在的户口子所在地（县/区编码）
wb201:孩子父母最近非假期的1个月，孩子平均每周能见到父母几次
wb501:孩子几个月大的时候开始自己走路
wc1:上个月孩子生病了几次
wc101:上个月孩子因病去医院看病几次
wc4:去年，孩子因病去医院看病几次
wc7:去年，因孩子患病，一共花了多少（元）
wc701:孩子患病的所有花费中，家里支付了多少（元）
wc802:去年，家里为孩子另外购买商业医疗保险花费了多少（元）
wb4:去年，父母双方都不与孩子在一起居住的连续时间最长为(周)
wb601:孩子几个月大的时候开始说完整的句子，如“我要吃饭”
wb701:孩子几个月大的时候开始能数1-10的
wb801:孩子几个月大的时候开始能独立小便
wc2:孩子在1岁以前共生病几次
wc201:孩子在1岁以前，因病去医院/医疗场所看了几次病
wd501:去年全年，这个孩子的所有教育支出（元）：学杂费
wd502:去年全年，这个孩子的所有教育支出（元）：书本费
wd503:去年全年，这个孩子的所有教育支出（元）：课外辅导/家教费
wd504:去年全年，这个孩子的所有教育支出（元）：住宿费
wd505:去年全年，这个孩子的所有教育支出（元）：交通费
wd506:去年全年，这个孩子的所有教育支出（元）：其他费用
wd5total:去年全年，这个孩子的所有教育支出总计（元）
wd510:去年全年，这个孩子的所有教育支出中，您家庭支付了多少（元）
wg101:最近

### 提取location列和关心的数据列

In [6]:
loca = 'provcd'
var = 'wm302'

### 查看提取列的基本信息

In [7]:
info = cfps[year][table].schema[var]
lc = cfps[year][table].schema[loca]
info,lc

({'type': 'int32',
  'key': '你觉得自己有多幸福',
  'range': [-8, 5, 4, 1, 3, 2, -2, -1],
  'details': [],
  'minmax': [-8.0, 5.0]},
 {'type': 'enum',
  'key': '省国标码',
  'range': {'-10': '无法判断',
   '-9': '缺失',
   '-8': '不适用',
   '-2': '拒绝回答',
   '-1': '不知道',
   '11': '北京市',
   '12': '天津市',
   '13': '河北省',
   '14': '山西省',
   '15': '内蒙古自治区',
   '21': '辽宁省',
   '22': '吉林省',
   '23': '黑龙江省',
   '31': '上海市',
   '32': '江苏省',
   '33': '浙江省',
   '34': '安徽省',
   '35': '福建省',
   '36': '江西省',
   '37': '山东省',
   '41': '河南省',
   '42': '湖北省',
   '43': '湖南省',
   '44': '广东省',
   '45': '广西壮族自治区',
   '46': '海南省',
   '50': '重庆市',
   '51': '四川省',
   '52': '贵州省',
   '53': '云南省',
   '54': '西藏自治区',
   '61': '陕西省',
   '62': '甘肃省',
   '63': '青海省',
   '64': '宁夏回族自治区',
   '65': '新疆维吾尔自治区'},
  'details': [],
  'minmax': [11.0, 62.0]})

#### 数据处理

In [8]:
condi = f'{var}>0 and {loca}>0'

In [9]:
df = sql(f'select {loca},{var} from {table}_{year} where {condi}')

In [10]:
# 各省对应变量数据加和取平均
data = df.groupby(loca,as_index=False).mean()

In [12]:
title = info['key']
axis = []
for i in data[loca]:
    name = lc['range'][str(i)]
    if name.endswith("省") or name.endswith("市"):name = name[:-1]
    else: name = {
        "广西壮族自治区": "广西",
        "内蒙古自治区": "内蒙古",
        "新疆维吾尔自治区": "新疆",
        "西藏自治区": "西藏"
    }[name]
    index = np.argwhere(data[loca].values==i)[0][0]
    axis.append([name,'%.02f'%(data[var][index])])
# da = data[loca].values
# print(np.argwhere(da ==62)[0][0])

In [13]:
(
    Map()
    .add(title, axis,maptype = "china")
    .set_series_opts(label_opts=opts.LabelOpts(is_show=False))
    .set_global_opts(title_opts=opts.TitleOpts(title=title),visualmap_opts=opts.VisualMapOpts(max_=max(data[var]), min_=min(data[var])))
    .render_notebook()
)